<a href="https://colab.research.google.com/github/DhanvinS/ML_Models-Supervised-and-Unsupervised/blob/main/Decision%20Trees%20%26%20Random%20Forests/Decision_Tree_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [52]:
from google.colab import files
files = files.upload()

Saving Data_Iris.csv to Data_Iris.csv


In [54]:
import pandas as pd
import numpy as np

df = pd.read_csv('Data_Iris.csv')
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species,species_name
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa


In [55]:
X = df[[
    'sepal length (cm)',
    'sepal length (cm)',
    'petal length (cm)',
    'petal width (cm)'
]].values
y = df['species'].values

In [59]:
X.shape

(150, 4)

In [58]:
y.shape

(150,)

In [61]:
# A function for Gini Impurity
def gini_impurity(y):

  classes, counts = np.unique(y, return_counts= True)
  impurity = 1.0

  for count in counts:
    p = count/len(y)
    impurity -= p**2
  return impurity

In [62]:
# a function to split our dataset
def split_dataset(X, y, feature_index, threshold):
  left_indices = X[: , feature_index] <= threshold
  right_indices = X[: , feature_index] > threshold

  return X[left_indices], y[left_indices], X[right_indices], y[right_indices]

In [63]:
def best_split(X, y):
  best_feature = None
  best_threshold = None
  best_gini = float("inf")

  n_samples, n_features = X.shape

  for feature in range(n_features):
    thresholds = np.unique(X[:, feature])

    for t in thresholds:
      X_left, y_left, X_right, y_right = split_dataset(X, y, feature, t)

      if len(y_left) == 0 or len(y_right) == 0:
        continue

      g_left = gini_impurity(y_left)
      g_right = gini_impurity(y_right)

      w_left = len(y_left) / len(y)
      w_right = len(y_right)/ len(y)

      gini = w_left * g_left + w_right * g_right

      if gini < best_gini:
        best_gini = gini
        best_feature = feature
        best_threshold = t

  return best_feature, best_threshold


In [64]:
class Node:
  def __init__(self, feature = None, threshold = None, left = None, right = None, value = None):
    self.feature = feature
    self.threshold = threshold
    self.left = left
    self.right = right
    self.value = value

In [65]:
def build_tree(X, y, depth = 0, min_samples_split = 2):

  if len(np.unique(y) == 1):
    return Node(value = y[0])

  if len(y) < min_samples_split:
    values, counts = np.unique(y, return_counts=True)
    return Node(value=values[np.argmax(counts)])

  feature, threshold = best_split(X, y)

  if feature is None:
        # Leaf node
        values, counts = np.unique(y, return_counts=True)
        return Node(value=values[np.argmax(counts)])

  X_left, y_left, X_right, y_right = split_dataset(X, y, feature, threshold)
  left_child = build_tree(X_left, y_left, depth+1)
  right_child = build_tree(X_right, y_right, depth+1)

  return Node(feature, threshold, left_child, right_child)


In [66]:
def predict_one(x, node):
    if node.value is not None:
        return node.value

    if x[node.feature] <= node.threshold:
        return predict_one(x, node.left)
    else:
        return predict_one(x, node.right)

def predict(X, tree):
    return np.array([predict_one(sample, tree) for sample in X])


In [67]:
tree = build_tree(X, y)
print("Decision tree trained successfully!")

Decision tree trained successfully!


In [70]:
sample = np.array([[1, 0.45, 1.4, 5]])  # one of your rows
prediction = predict(sample, tree)[0]

print("Predicted class:", prediction)

Predicted class: 0
